---
- dir, help, print(sklearn.__all__) 사용하기

- 분류 모델: LogisticRegression, RandomForestClassifier, XGBClassifier
- 분류 검증: (predict) f1_score, recall, precision, accuracy_score /(proba)roc_auc_score, 
- 회귀 모델: LinearRegression, RandomForestRegression, XGBRegressor
- 회귀 검증: (predict) r2_score, mean_absolute_error
---

#1. 작업 1유형

1) 정리본

In [ ]:
#기본
df.loc[:,'rank'] = list(range(1,101))*10
df['B'] = df.A.str.len()
pd.merge(A, B, how='left', on='columns')
df = df['A'].fillna(method = 'ffill')  #★★★★★
drop_duplicates('columns1', keep='last')  #마지막추출
dic = { x:y for x,y in mean.items()}
---------------------------------------------------------------------------------------
#시간
df['ct'] = pd.to_datetime(df['ct'])   #선언
df.loc[df.ct <= pd.to_datetime('2021-11-01')]                #'=='경우 시간이 다 맞아야 함
df.loc[df.ct.dt.strftime('%Y-%m-%d %H') == '2021-11-01 15']  #시간 일부 ok
df.loc[df.ct.dt.day_name() == 'Sunday']   #Sunday 대문자로
df.loc[df.ct.dt.second == 21]             #연월일시분초
import datetime
df.loc[df.ct.diff() <= datetime.timedelta(minutes=5))] #t시간차
---------------------------------------------------------------------------------------
#표로 만들기
df.groupby(['A','B'], as_index = False).size()  #as_index해야 DataFrame유지.
df.groupby(['A']).size().to_frame('B')  #위랑 다르게 컬럼명 지정하면서 좌-우 대각선 배치
df.rename(columns={'전 이름':'바꿀 이름'})

df.pivot(index='A', columns='B')
---------------------------------------------------------------------------------------
#검정
#샤피로검정 : x의 정규성 검정
from scipy.stats import shapiro
shapiro(x)[1]
#카이제곱 : x,y가 독립인지 카이제곱검정으로 확인.
from scipy.stats import chi2_contingency
chi2, p, dof, expected = chi2_contingency(pd.crosstab(x,y))
#랜포
pd.DataFrame({'importance':rf.feature_importances_},index=x.columns).sort_values('importance',ascending=False)
---------------------------------------------------------------------------------------

2) 오답

In [ ]:
import pandas as pd
x_train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
x_test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv')
submission= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv')

#display(x_train.head())
#display(x_test.head())
#display(submission.head())



#1. ESD
import warnings
warnings.filterwarnings('ignore')
#print(x_train.shape)  / 행열개수
#print(x_test.shape)

#print(x_train.isnull().sum()) / 결측치
#print(x_test.isnull().sum())

#print(x_train.info()) / 데이터타입
#print(x_test.info())

#for i in x_test.select_dtypes(include=object): / 이상값확인
#  print(x_test[i].value_counts())



#2. x,y 정리

#결측치 처리
#x_train = x_train['age'].fillna(method = 'ffill')  #★★★★★
#x_train = x_train['age'].fillna(x_train['age'].median())
#x_train = x_train.dropna()
#x_train = x_train.drop(columns='', index='')

#이상값처리 및 데이터타입 변경
#x_train.age.str.replace.('*','')
#x_train.astype('int')

#x,y 지정
x_test_id = x_test['ID']
y_train = x_train['y'].map(lambda x: 1 if x == 'yes' else 0)
x_train = x_train.iloc[:, 1:-1]
x_test = x_test.iloc[:,1:]

cols_ob = x_train.select_dtypes(include=object).columns
cols_num = x_train.select_dtypes(exclude=object).columns

#print(x_train.shape, x_test.shape)



#3. 라벨인코딩
from sklearn.preprocessing import LabelEncoder
for i in cols_ob :
  le = LabelEncoder()
  x_train[i] = le.fit_transform(x_train[i])
  x_test[i] = le.transform(x_test[i])

#print(x_train.shape, x_test.shape)


#4. 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train[cols_num] = scaler.fit_transform(x_train[cols_num])
x_test[cols_num] = scaler.transform(x_test[cols_num])

x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)

#print(x_train.shape, x_test.shape)


#5. 홀드아웃
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, random_state=2022, test_size=0.33, stratify=y_train)

#print(x_train.shape,x_tr.shape, x_test.shape)



#6. 모델링 
#분류모델
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
#회귀모델
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
#분류예측값
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
#분류예측확률값
from sklearn.metrics import roc_auc_score
#회귀예측값
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


lr = LogisticRegression()
lr.fit(x_tr, y_tr)
#예측값
pred1 = lr.predict(x_val)
print(f1_score(y_val, pred1))
print(accuracy_score(y_val, pred1))
print(recall_score(y_val, pred1))
print(precision_score(y_val, pred1))
print('----------------------------')
#예측확률값
pred1_h = lr.predict_proba(x_val)[:,1]
print(roc_auc_score(y_val, pred1_h))
print('----------------------------')

rf = RandomForestClassifier()
rf.fit(x_tr, y_tr)
#예측값
pred2 = rf.predict(x_val)
print(f1_score(y_val, pred2))
print(accuracy_score(y_val, pred2))
print(recall_score(y_val, pred2))
print(precision_score(y_val, pred2))
print('----------------------------')
#예측확률값
pred2_h = rf.predict_proba(x_val)[:,1]
print(roc_auc_score(y_val, pred2_h))
print('----------------------------')

xg = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)  #★★★★
xg.fit(x_tr, y_tr)
#예측값
pred3 = xg.predict(x_val)
print(f1_score(y_val, pred3))
print(accuracy_score(y_val, pred3))
print(recall_score(y_val, pred3))
print(precision_score(y_val, pred3))
print('----------------------------')
#예측확률값
pred3_h = xg.predict_proba(x_val)[:,1]
print(roc_auc_score(y_val, pred3_h))
print('----------------------------')


#7. 결과
xg.fit(x_train, y_train) #★★★★★★
#예측값
pred = xg.predict(x_test)
#예측값확률
pred_h = xg.predict_proba(x_test)[:,1]
result = pd.DataFrame({'ID':x_test_id, 'predict': pred}).to_csv('00000.csv', index=False)

0.34517766497461927
0.7266949152542372
0.23376623376623376
0.6594827586206896
----------------------------
0.7240399454037546
----------------------------
0.5527306967984933
0.7763653483992468
0.4484339190221543
0.7202453987730061
----------------------------
0.7806294839999782
----------------------------
0.559610705596107
0.7869585687382298
0.439266615737204
0.7707774798927614
----------------------------
0.7857054480055501
----------------------------


In [ ]:
#https://www.datamanim.com/dataset/03_dataq/typeone.html

★#15) 공범 EP1의 비디오정보 데이터중 수집간격이 5분 이하, 20분이상인 데이터 구간( 해당 시점 전,후) 의 시각을 모두 출력하라
#import pandas as pd
#df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')
import datetime
ep_one = video.loc[video.videoname.str.contains('1')].sort_values('ct').reset_index(drop=True)
ep_one[(ep_one.ct.diff(1) >=datetime.timedelta(minutes=20)) | (ep_one.ct.diff(1) <=datetime.timedelta(minutes=5))]
ep_one[ep_one.index.isin([720,721,722,723,1635,1636,1637])]

★#16) 2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임으로 만드시오
#import pandas as pd
#df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')
start_date = video.sort_values(['ct','videoname']).drop_duplicates('videoname')[['ct','videoname']]
start_date['date'] = start_date.ct.dt.date
start_date[['date','videoname']]

★#23) 년도 표기가 4자리 숫자로 안된 케이스가 존재한다. 해당 건은 몇건인지 출력하라
#import pandas as pd
#df= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/worldcup/worldcupgoals.csv')
df['yearLst'] = df.Years.str.split('-')
def checkFour(x):
    for value in x:
        if len(str(value)) != 4:
            return False       
    return True
df['check'] = df['yearLst'].apply(checkFour)
len(df[df.check ==False])

★#45) 2019년도 데이터들만 추출하여 각변수간 상관계수를 구하고 내림차순으로 ㅔ정렬한 후 상위 5개를 데이터 프레임으로 출력하라. 컬럼명은 v1,v2,corr으로 표시하라
#import pandas as pd
#df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv',encoding='utf-8')
zz = df[df.년도 ==2019].corr().unstack().to_frame().reset_index().dropna()
result = zz[zz[0] !=1].sort_values(0,ascending=False).drop_duplicates(0)
answer = result.head(5).reset_index(drop=True)
answer.columns = ['v1','v2','corr']
display(answer)

#50) 2018년,2019년 모두 기록이 있는 나라들 중 년도별 행복점수가 가장 증가한 나라와 그 증가 수치는?
#import pandas as pd
#df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv',encoding='utf-8')
count = df.나라명.value_counts()
target = count[count>=2].index

df2 =df.copy()
multiple = df2[df2.나라명.isin(target)].reset_index(drop=True)
multiple.loc[multiple['년도']==2018,'점수'] = multiple[multiple.년도 ==2018]['점수'].values * (-1)
multiple.groupby('나라명').sum()['점수'].sort_values().to_frame().iloc[-1]


#실2-4) bmi컬럼의 각 결측치들을 결측치를 가지지 않는 환자 나이대(10단위)의 평균 bmi 값으로 대체한 후 대체된 bmi 컬럼의 평균을 구하여라
#import pandas as pd
#df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')
#df['age'] = df['age'].str.replace('*','').astype('int')
mean = df[df.bmi.notnull()].groupby(df.age//10 *10).bmi.mean()
dic = { x:y for x,y in mean.items()}
df.loc[df.bmi.isnull(),'bmi']  =(df[df.bmi.isnull()].age//10*10).map(lambda x : dic[x])
df.bmi.mean()

#2. 작업2유형 